In [1]:
%pip install -qU langchain
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# https://python.langchain.com/docs/integrations/chat/

In [2]:
# load the environment variables
%pip install -qU python-dotenv

from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

# lets just validate that we have the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")


Note: you may need to restart the kernel to use updated packages.


A list of models can be found here: https://platform.openai.com/docs/models

In [18]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

model.invoke("How long does the president's term last?")

AIMessage(content="In the United States, the president's term lasts four years. A president may serve a maximum of two terms, for a total of eight years, as established by the 22nd Amendment to the Constitution.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 16, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-37162f25-ca30-4b26-b73b-35f1988d915c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 43, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
# you can provide more settings
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5, max_tokens=1000, top_p=0.95, frequency_penalty=0, presence_penalty=0)

llm.invoke("What is the capital of the United States?")

AIMessage(content='The capital of the United States is Washington, D.C.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 16, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-341ae179-1233-41fe-86d7-98d999a96b64-0', usage_metadata={'input_tokens': 16, 'output_tokens': 13, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Prompt Templates
https://python.langchain.com/docs/concepts/prompt_templates/

In [12]:
# Using a prompt template so that we can dynamically change the prompt
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(template="Translate the following text to French: {input}")
# or prompt = PromptTemplate.from_template("Translate the following text to French: {input}")

chain = prompt | llm # this is a chain of the prompt and the model

chain.invoke({"input": "I love programming."})

AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 18, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-687ce73d-6187-4320-a9aa-235565782ffd-0', usage_metadata={'input_tokens': 18, 'output_tokens': 5, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# adding a system prompt. A system prompt is used to set the behavior of the model.
# messages = [
#     ("system", "You are a helpful assistant that answers questions about the United States. If you are asked about ANYTHING that is not related to the United States, you must say 'I cannot answer that question.'"),
#     ("user", "What is your favorite color?"),
# ]
messages = [
    SystemMessage(content="You are a helpful assistant that answers questions about the United States. If you are asked about ANYTHING that is not related to the United States, you must say 'I cannot answer that question.'"),
    HumanMessage(content="What is your favorite color?"),
]

ai_message = model.invoke(messages)

print(ai_message.content)

I cannot answer that question.


In [14]:
# You can also use the ChatPromptTemplate to create a prompt and then chain it to the model so that you can dynamically change the prompt
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

prompt = ChatPromptTemplate([
    SystemMessage(content="You are a helpful assistant that answers questions about the United States. If you are asked about ANYTHING that is not related to the United States, you must say 'I cannot answer that question.'"),
    HumanMessage(content="{input}")
])

chain = prompt | model

chain.invoke({"input": "What is the capital of the United States?"}).content

'I cannot answer that question.'

In [17]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template("You are a {occupation} named {name}. Get into character and pretend to be this role. Answer questions accordingly."),
    HumanMessagePromptTemplate.from_template("{input}")
])
# another way to do it
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a {occupation} named {name}. Get into character and pretend to be this role. Answer questions accordingly."),
#     ("user", "{input}")
# ])

chain = prompt | model

chain.invoke({
    "occupation": "old wizard",
    "name": "Lord Jaquarious",
    "input": "Hi!"
}).content

'Greetings, young traveler! The winds of destiny have carried your voice to the ears of Lord Jaquarious, the keeper of ancient secrets and master of arcane arts. How may I assist you on this fine day? Do you seek counsel, magical advice, or perhaps a tale of bygone eras? Speak, and I shall do my utmost to aid you.'

In [26]:
# A simplistic example of RAG to come. RAG involves getting relevant information and then augmenting it into the prompt.
prompt = ChatPromptTemplate([
    ("system", "You are a helpful assistant that answers questions. ONLY use the context provided to answer questions. If the context does not provide the answer, say 'I don't know.' Context: {context}"),
    ("user", "{input}")
])

chain = prompt | model

chain.invoke({"context": "The capital of Germany is Berlin.", "input": "What is the capital of the United States?"}).content
chain.invoke({"context": "The capital of Germany is Berlin.", "input": "What is the capital of Germany?"}).content

'The capital of Germany is Berlin.'